# **Keras 예제 - Seq2Seq 로 사칙연산 구현(연산자 2개)**

출처
- RNN 덧셈 예제 : https://github.com/keras-team/keras/blob/2.0.0/examples/addition_rnn.py
- 사칙연산 예제 : https://towardsdatascience.com/making-rnn-model-learn-arithmetic-operations-b016ec4d8388
- 텐서플로우 버전 : 2.1.0



In [3]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)
Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.
Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

'An implementation of sequence to sequence learning for performing addition\nInput: "535+61"\nOutput: "596"\nPadding is handled by using a repeated sentinel character (space)\nInput may optionally be inverted, shown to increase performance in many tasks in:\n"Learning to Execute"\nhttp://arxiv.org/abs/1410.4615\nand\n"Sequence to Sequence Learning with Neural Networks"\nhttp://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf\nTheoretically it introduces shorter term dependencies between source and target.\nTwo digits inverted:\n+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs\nThree digits inverted:\n+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs\nFour digits inverted:\n+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs\nFive digits inverted:\n+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs\n'

In [1]:
from __future__ import print_function
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
from six.moves import range
import matplotlib.pyplot as plt

In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    # 초기화 : 사용되는 문자 집합이 주어지면 caharacter table 을 초기화한다.
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        # 문자 집합이 주어지면 각 문자에 대한 인덱스를 매긴다.
        # char_indices : (문자, 인덱스), indices_char : (인덱스, 문자)
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        print(self.char_indices)
        print(self.indices_char)

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        # 문장(C)의 문자에 해당하는 행렬 위치를 0->1 로 바꾼다.
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1    # 순서대로 i번째 행에 char_indices[c] 열에 1 대입
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
def weird_division(n, d):
    return n / d if d else 0

In [5]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 2
INVERT = True

# Maximum length of input is 'int + int * int' (e.g., '34+67*2' or '50-2/14'). 
# Maximum length of int is DIGITS.
MAxLEN = DIGITS + 1 + DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
# ctable : 문자 집합에 대해 character table 을 만든 인스턴스(?)
chars = '0123456789+*/-. '
calc = '+*/-'
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
# file_path = 'datset/3digit_1arith.txt'

# print('Get data...from txt')
# for line in open(file_path, 'r'):
#         idx = line.find('_')
#         questions.append(line[:idx][::-1])
#         expected.append(line[idx+1:-1])

# Generating Data...
np.random.randint(1, 20) : 1~19까지 랜덤한 숫자 1개
while len(questions) < TRAINING_SIZE:   #50000개 만듦
    # f : 최대 2자리까지 랜덤한 숫자를 만든다.
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b, c = f(), f(), f()

    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b, c)))
    if key in seen:
        continue
    seen.add(key)

    # Pad the data with spaces such that it is always MAxLEN.
    # q = a + b / c 또는 a * b - c
    q = str(a) + np.random.choice(list(calc)) + str(b) + np.random.choice(list(calc)) + str(c)
    query = q + ' ' * (MAxLEN - len(q)) # 전체 길이 - q 길이 만큼 padding 을 줘서 전체 길이가 맞춰지도록.
    if '+' in q:
      ans = str(a + b)
    elif '-' in q:
      ans = str(a - b)
    elif '*' in q:
      ans = str(a * b)
    else:
      res = weird_division(a, b)
      ans = str(round(res,2))
  
    # Answers can be of maximum size DIGITS + 1.
    # answer 도 패딩을 맞춰준다. (answer가 될 수 있는 최대 길이에서 - 현재 나온 답의 길이 만큼)
    # answer 의 최대 길이 : 6자리(999*999 = 998001)
      ans += ' ' * (DIGITS + 3 - len(ans))

    # Input의 Reverse 여부
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1] # Reverse
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))


{' ': 0, '*': 1, '+': 2, '-': 3, '.': 4, '/': 5, '0': 6, '1': 7, '2': 8, '3': 9, '4': 10, '5': 11, '6': 12, '7': 13, '8': 14, '9': 15}
{0: ' ', 1: '*', 2: '+', 3: '-', 4: '.', 5: '/', 6: '0', 7: '1', 8: '2', 9: '3', 10: '4', 11: '5', 12: '6', 13: '7', 14: '8', 15: '9'}
Get data...from txt


In [6]:
# Reverse 해서 인풋이 거꾸로 출력됨.
print(questions[1] + '?' + expected[1])
print(questions[1][::-1])
print(expected[1])

  84*13?1488
31*48  
1488


In [18]:
# f = open('3digit_1arith.txt', 'w')
# for i in range(len(questions)):
#     data = questions[i][::-1] + '_' + expected[i] + '\n'
#     f.write(data)
# f.close()


In [7]:
print('Vectorization...')
# np.zeros(shape, dtype, order)
x = np.zeros((len(questions), MAxLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 3, len(chars)), dtype=np.bool)
print(x.shape)
print(y.shape)

Vectorization...
(50000, 7, 16)
(50000, 6, 16)


In [8]:
print('Vectorization...')
# np.zeros(shape, dtype, order)
# x : (50000, 7, 15), y : (50000, 6, 15)
# 입력데이터를 Encode
x = np.zeros((len(questions), MAxLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 3, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAxLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 3)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
# x의 뒷부분이 더 커지기 때문에 (x, y) 를 섞는다. (???) -> 어쨋든 셔플
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
# Test Set : 0~45000, Validation Set : 45000~50000
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Vectorization...
Training Data:
(45000, 7, 16)
(45000, 6, 16)
Validation Data:
(5000, 7, 16)
(5000, 6, 16)


In [9]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()

# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAxLEN, len(chars))))

# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 3))

# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    # return_sequences(시퀀스 출력 여부): True(각 시퀀스에서 출력, many-to-many 일 때)
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
# TimeDistributed :7개의 시간 단계 각각에 독립적으로 Dense 레이어를 적용
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               74240     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 6, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 6, 16)             2064      
_________________________________________________________________
activation (Activation)      (None, 6, 16)             0         
Total params: 207,888
Trainable params: 207,888
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model each generation and show predictions against the validation
# dataset.
acc_list = []
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1,
              validation_data=(x_val, y_val))
    # Select samples from the validation set at random so we can visualize
    # errors.
    correct_num = 0
    for i in range(len(x_val)):
        rowx, rowy = x_val[np.array([i])], y_val[np.array([i])]
        # predict_classes : 0 or 1로 출력 (predict 와 약간 다름) --> 2.6버전에서 삭제됨
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        
        if guess == correct:
          correct_num += 1
        else:
          correct_num += 0
        # print('guess: ', guess)
        # print('Q', q[::-1] if INVERT else q)
        # print('T', correct)
        # if correct == guess:
        #     print(colors.ok + '☑' + colors.close, end=" ")
        # else:
        #     print(colors.fail + '☒' + colors.close, end=" ")
        # print(guess)
        # print('---')

    acc = float(correct_num) / len(x_val)
    acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))
    if iteration == 100:
      print('100번째 정확도 : ', acc)
model.save('3digit_1arith.h5')


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
45000/45000 [==============================] - 15s 331us/sample - loss: 1.4405 - accuracy: 0.2402 - val_loss: 1.3092 - val_accuracy: 0.2372
검증 정확도 0.940%

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
45000/45000 [==============================] - 13s 278us/sample - loss: 1.2705 - accuracy: 0.2453 - val_loss: 1.2482 - val_accuracy: 0.2451
검증 정확도 1.320%

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
45000/45000 [==============================] - 12s 278us/sample - loss: 1.2311 - accuracy: 0.2540 - val_loss: 1.2116 - val_accuracy: 0.2524
검증 정확도 0.920%

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
45000/45000 [==============================] - 12s 278us/sample - loss: 1.2015 - accur

In [ ]:
# 그래프 그리기
x = np.arange(len(acc_list))
plt.plot(x, acc_list, marker='o')
plt.xlabel('에폭')
plt.ylabel('정확도')
plt.ylim(0, 1.0)
plt.show()